In [21]:
# df_adjOHLCV has data only for NYSE trading days, no weekend data
# df_adjOHLCV includes data for weekends when BTC trades
# read symbols in file to list syms_in_file
# download OHLCV data for symbols in syms_in_file
# drop symbols with all NaN in OHLCV columns from df
# rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# drop weekend data by reindex to date index of index_symbol
# pickled df_adjOHLCV
# pickled df_adjOHLCV
# pickled symbols_df_adjOHLCV
# create df_symbols_close, sort df by symbols
# pickled df_symbols_close

In [22]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
from random import randint
from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA

# from myUtils import drop_symbols_all_NaN, yf_symbols_close, chunked_list # NOQA
from myUtils import drop_symbols_all_NaN, yf_symbols_close_1, chunked_list # NOQA

from myUtils import yf_download_AdjOHLCV, yf_download_AdjOHLCV_noAutoAdj
verbose = False  # True prints more output
# verbose = True  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# filename_symbols = path_data_dump + 'symbols_mktCap_2b.csv'  # symbols text file
filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in adjOHLCV

In [23]:
# function to adjust OHLCV
# https://github.com/ranaroussi/yfinance/issues/687
def auto_adjust(data):
    df = data.copy()
    ratio = df["Close"] / df["Adj Close"]
    df["Adj Open"] = df["Open"] / ratio
    df["Adj High"] = df["High"] / ratio
    df["Adj Low"] = df["Low"] / ratio

    df.drop(
        ["Open", "High", "Low", "Close"],
        axis=1, inplace=True)

    df.rename(columns={
        "Adj Open": "Open", "Adj High": "High",
        "Adj Low": "Low", "Adj Close": "Close"
    }, inplace=True)

    # df = df[["Open", "High", "Low", "Close", "Volume"]]
    return df[["Open", "High", "Low", "Close", "Volume"]]

In [ ]:
# Stop if last date in df_adjOHLCV is the same as Yahoo download
index_symbol = "XOM"  
df_XOM = yf.download(index_symbol)
download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
df_adjOHLCV_last_date = df.index[-1].strftime('%Y-%m-%d')
if download_last_date == df_adjOHLCV_last_date:
  msg_stop = f"df_adjOHLCV is up-to-date,\nlast date from df_adjOHLCV is {df_adjOHLCV_last_date},\nlast date from Yahoo's {index_symbol} download is {download_last_date}"
  raise SystemExit(msg_stop )

In [94]:
# verify df test_symbols' close against Yahoo
test_symbols = ['A', 'SHEL', 'YUM']
_df_list=[]
_df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)
_df_list.append(_df)
_df = pd.concat(_df_list, axis=1)
# get unique symbols from column name
_l_symbols = list(set(_df.columns.get_level_values(0)))
_l_symbols.sort()

_df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  _df1 = auto_adjust(_df[symbol])
  _df_adj_list.append(_df1)

_df_adj = pd.concat(_df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
_df_adj.columns = _col_names

for symbol in _l_symbols:

  s = _df_adj.iloc[-222]
  # s = df_adjOHLCV.iloc[-222]  # use to check download after FINISHED running all cells
  
  sDate = s.name.strftime('%Y-%m-%d')
  sClose = s[symbol].Close

  df_sym = yf.Ticker(symbol).history(period='2y')
  # 10:45PM got Adj Close
  yhClose = df_sym.loc[sDate]['Close']

  abs_pct_diff = abs(1 - sClose/yhClose)*100
  print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
  Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
  if abs_pct_diff > .0001:
    msg_err = f'{symbol}  %_dif_Close > .0001'
    # raise SystemExit(msg_err)
    print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
  if symbol == test_symbols[-1]:
    msg_done = f"No errors found.  Test_symbols' 'Adjusted Close' matched Yahoo symbols' Close "
    print(msg_done)

[*********************100%***********************]  3 of 3 completed
symbol:     A   Date: 2021-12-22   df_Close:  155.78232   Yahoo_Close:  155.78232   %_dif_Close: 0.00000
symbol:  SHEL   Date: 2021-12-22   df_Close:   43.52000   Yahoo_Close:   42.74147   %_dif_Close: 1.82149
SHEL  %_dif_Close > .0001
symbol:   YUM   Date: 2021-12-22   df_Close:  132.69643   Yahoo_Close:  132.69643   %_dif_Close: 0.00000
No errors found.  Test_symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [26]:
# read symbols in file to a list and convert to chunks
# TODO: each chunk must have more than 1 symbol, otherwise, symbol won't appear as column name
symbols_in_file = read_symbols_file(filename_symbols)
symbols_chunks = chunked_list(symbols_in_file, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]

In [27]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(symbols_chunks):
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
  symbols_download_err.append(list(shared._ERRORS.keys()))
  print(f'symbols_download_err: {symbols_download_err}')
  df_list.append(df)
  # pause between download
  if i < len(symbols_chunks) - 1 :  # skip pause after last download
    print(f'downloaded symbols from chuck {i}, sleep start')
    # sleep 78(18m 25s)
    time.sleep(78)
    print(f'downloaded symbols from chuck {i}, sleep ends')
  else:
    print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

[*********************100%***********************]  400 of 400 completed
symbols_download_err: [[]]
downloaded symbols from chuck 0, sleep start
downloaded symbols from chuck 0, sleep ends
[*********************100%***********************]  400 of 400 completed
symbols_download_err: [[], []]
downloaded symbols from chuck 1, sleep start
downloaded symbols from chuck 1, sleep ends
[*********************100%***********************]  400 of 400 completed

2 Failed downloads:
- EMP: No data found for this date range, symbol may be delisted
- EAI: No data found for this date range, symbol may be delisted
symbols_download_err: [[], [], ['EMP', 'EAI']]
downloaded symbols from chuck 2, sleep start
downloaded symbols from chuck 2, sleep ends
[*********************100%***********************]  400 of 400 completed
symbols_download_err: [[], [], ['EMP', 'EAI'], []]
downloaded symbols from chuck 3, sleep start
downloaded symbols from chuck 3, sleep ends
[*********************100%*******************

In [28]:
# flatten symbols_download_err which is a list-of-lists
l_symbols_err = [val for sublist in symbols_download_err for val in sublist]
l_symbols_err

['EMP', 'EAI', 'SFB']

In [29]:
df = pd.concat(df_list, axis=1)
df.drop(l_symbols_err, axis=1, level=0, inplace=True)  # drop symbols with errors  
l_symbols = list(set(df.columns.get_level_values(0)))  # get unique symbols
l_symbols.sort()
df = df[l_symbols]  # sort columns by list
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
pickle_dump(df, path_data_dump, filename_pickled_df_OHLCVA_downloaded)  # save OHLCA data

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [80]:
# df = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded)

In [81]:
# adjust OHLC
df_adj_list=[]
for symbol in l_symbols:
  _df = auto_adjust(df[symbol])
  df_adj_list.append(_df)

df_adj = pd.concat(df_adj_list, axis=1)
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
col_names = pd.MultiIndex.from_product([l_symbols, cols])  # create multilevel column names
df_adj.columns = col_names  # reindex columns

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adj, path_data_dump, filename_pickled_df_adjOHLCV)  # save adjusted OHLCV data

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [82]:
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV)

In [83]:
# drop symbols with all NaN in OHLCV columns from df
df_adjOHLCV, symbols_OHLCV, symbols_dropped = drop_symbols_all_NaN(df_adjOHLCV, verbose)
print(f'symbols with all NaN dropped from df_adjOHLCV: {symbols_dropped}')

Symbol's OHLCV are all NaN: []
symbols with all NaN dropped from df_adjOHLCV: []


In [84]:
# rename columns OHLCV *ONLY AFTER* dropping symbols with all NaN from df,
#   symbols with all NaN has an added AdjClose column and will cause errors  
#  rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
#  .remove_unused_levels() prevents ValueError
#   e.g ValueError: On level 1, code max (5) >= length of level (5). NOTE: this index is in an inconsistent state
# The error may be caused by removing symbols from the dataframe with all NaN in OHLCV columns
df_adjOHLCV.columns = df_adjOHLCV.columns.remove_unused_levels()
# # set_levels reorders df columns in alphabetical order, so the list of column names also needs to be in alphabetical order
# df_adjOHLCV.columns = df_adjOHLCV.columns.set_levels(['close', 'high', 'low', 'open', 'volume'], level=1)

In [85]:
# drop weekend data by re-indexing to date-index of index_symbol
myNaN = float('nan')
# use Exxon's date as proxy for NYSE trading dates
df_adjOHLCV = df_adjOHLCV.reindex(df_XOM.index, fill_value=myNaN)

In [86]:
# pickle df_adjOHLCV and symbols
# print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
# pickle_dump(df_adjOHLCV_downloaded, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adjOHLCV, path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
pickle_dump(symbols_OHLCV, path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [87]:
symbols_OHLCV = list(set([i[0] for i in list(df_adjOHLCV)]))
df_symbols_close = df_adjOHLCV.xs('Close', level=1, axis=1)
pickle_dump(df_symbols_close, path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

In [88]:
# retrieve pickled files

print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA_downloaded = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
symbols_df = pickle_load(path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [91]:
print(f'df_OHLCVA_downloaded.info():\n{df_OHLCVA_downloaded.info()}\n')
print(f'df_adjOHLCV.info():\n{df_adjOHLCV.info()}\n')
print(f'df_close.info():\n{df_close.info()}\n')
print(f'len(symbols_df):\n{len(symbols_df)}\n')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15320 entries, 1962-01-02 to 2022-11-08
Columns: 21900 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(21875), int64(25)
memory usage: 2.5 GB
df_OHLCVA_downloaded.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15320 entries, 1962-01-02 to 2022-11-08
Columns: 18250 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(18225), int64(25)
memory usage: 2.1 GB
df_adjOHLCV.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15320 entries, 1962-01-02 to 2022-11-08
Columns: 3650 entries, A to ZWS
dtypes: float64(3650)
memory usage: 426.7 MB
df_close.info():
None

len(symbols_df):
3650

